In [1]:
import pymysql
import pandas as pd 

In [4]:
data = pd.read_csv ("..\src\Coffee_Clean.csv",index_col=None)
data['Id'] = data.index


In [6]:
def CreateCoffeshop(df):
    sf = df[['Id','Name','Time-Open', 'Time-Close','Sum-Of-Featurs','Phone']]
    sf.rename(columns = {'Name':' CoffeeShopName','Phone':'PhoneNumber','Time-Open':'StartAt','Time-Close':'EndAt','Sum-Of-Featurs':'CountOfFeaturs'},inplace = True)
    return sf

In [7]:
def CreateFeatures(df):
    sf = df[['free internet', 'hookah', 'cigarettes', 'parking-lot', 'Pose',
       'Free-Delivery', 'View', 'BreakFast', 'Outdoor', 'Live-Music']]
    sf['CoffeID'] = df['Id']
    sf.rename(columns = {'free internet':'FreeInternet','hookah':'Hookah','cigarettes':'Cigarettes',
                         'parking-lot':'Parkinglot','Free-Delivery':'FreeDelivery','View':'CoffeView','Live-Music':' LiveMusic'})
    return sf
def CreateAddress(df):
    sf = df[['City', 'Region','Street']]
    sf['CoffeID'] = df['Id']
    sf.rename(columns = {'Region':'NeighborHood','Street':'AuxiliaryRoad'},inplace = True)
    return sf
def CreatePoints(df):
    sf = df[['Total-Rate', 'Quality-Rate','Service-Rate','Decoration-Rate','Worth-Ratio']]
    sf['CoffeID'] = df['Id']
    sf.rename(columns = {'Quality-Rate':'QualityPoint','Service-Rate':'ServicePoint','Decoration-Rate':'DecorationPoint','Total-Rate':
                          'TotalRate','Worth-Ratio':'WorthRatio'}, inplace = True)
    return sf

   

In [8]:
from sqlalchemy.engine import URL
#local host of your databaseP
url_object = URL.create(
    "mysql+pymysql",
    username="root",
    password="ehsan",
    host="localhost",    
)

In [9]:
import sqlalchemy as db
engine = db.create_engine(url_object)
conn = engine.connect()
engine.execute("CREATE DATABASE IF NOT EXISTS Doffeeshopdata") 
engine.execute("ALTER DATABASE coffeeshopdata CHARACTER SET utf8 COLLATE utf8_unicode_ci")
engine.execute("USE Doffeeshopdata") 

In [10]:
coffeeshop = CreateCoffeshop(data)
featuers = CreateFeatures(data)
address = CreateAddress(data)
points = CreatePoints(data)

C:\Users\Ehsan\AppData\Local\Temp\ipykernel_23728\425962471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf.rename(columns = {'Name':' CoffeeShopName','Phone':'PhoneNumber','Time-Open':'StartAt','Time-Close':'EndAt','Sum-Of-Featurs':'CountOfFeaturs'},inplace = True)
C:\Users\Ehsan\AppData\Local\Temp\ipykernel_23728\1891073458.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf['CoffeID'] = df['Id']
C:\Users\Ehsan\AppData\Local\Temp\ipykernel_23728\1891073458.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [11]:
coffeeshop.set_index('Id',inplace = True)
featuers.set_index('CoffeID',inplace = True)
address.set_index('CoffeID',inplace = True)
points.set_index('CoffeID',inplace = True)

In [12]:
coffeeshop.to_sql('coffeeshop',
               con = engine, if_exists = 'append')

1231

In [13]:
featuers.to_sql('featuers',
               con = engine, if_exists = 'append')

1231

In [14]:
address.to_sql('address',
               con = engine, if_exists = 'append')

1231

In [15]:
points.to_sql('points',
               con = engine, if_exists = 'append')

1231

In [16]:
d = pd.DataFrame(engine.execute("SELECT * FROM coffeeshop").fetchall())
d

Id       CoffeeShopName StartAt     EndAt  CountOfFeaturs  PhoneNumber
0        0                خامیز    8:00  24:00:00               2   2126808894
1        1  کافه رستوران بارمان   10:30  24:00:00               1   2122202929
2        2                سیزده    8:00  24:00:00               2   2188914292
3        3              اینساید   10:00  24:00:00               3   2122661143
4        4            هوبو لانژ   14:00  24:00:00               1   9190054735
...    ...                  ...     ...       ...             ...          ...
1226  1226            پنج و نیم    8:30  24:00:00               4   9017765438
1227  1227                ریتون    9:00  24:00:00               4     32469231
1228  1228              باریستا    7:30  24:00:00               3     32457711
1229  1229              کنج دنج    8:00  24:00:00               2   7633740333
1230  1230                پیانو   11:00  24:00:00               3   9133239334

[1231 rows x 6 columns]

In [17]:
d = pd.DataFrame(engine.execute("SELECT * FROM address").fetchall())
d

CoffeID      City        NeighborHood  \
0           0     تهران            زعفرانیه   
1           1     تهران             فرمانیه   
2           2     تهران        فلسطین شمالی   
3           3     تهران               فرشته   
4           4     تهران    بلوار سعادت آباد   
...       ...       ...                 ...   
1226     1226     کرمان   خیابان بهمنیار ۲۴   
1227     1227     کرمان      خیابان بهمنیار   
1228     1228     کرمان          بلوار حمزه   
1229     1229  بندرعباس         رسالت شمالی   
1230     1230  بندرعباس  خیابان رسالت جنوبی   

                                         AuxiliaryRoad  
0               آصف، انتهای کوچه حامد، نبش باغستان ششم  
1      خیابان لواسانی، مابین آقایی و وطن پور، پلاک 152  
2                       نرسیده به بلوار کشاورز،پلاک431  
3                                 مريم غربي،كوئين سنتر  
4                  نبش کوچه حق طلب،ساختمان تهران پاریس  
...                                                ...  
1226                              انتهای محوطه پارکینگ  
1227                                        بهمنیار 24  
1228                                  بین کوچه 24 و 26  
1229     چهارراه پردیس، ساختمان ثبت شرکت(ریاضیات خیام)  
1230                        بلوار12فروردین به طرف ۹ دی  

[1231 rows x 4 columns]

In [18]:
engine.execute("ALTER TABLE coffeeshop ADD PRIMARY KEY (Id) ")
engine.execute("ALTER TABLE featuers ADD FOREIGN KEY  (CoffeID) REFERENCES coffeeshop(Id) ")
engine.execute("ALTER TABLE address ADD FOREIGN KEY  (CoffeID) REFERENCES coffeeshop(Id) ")
engine.execute("ALTER TABLE points ADD FOREIGN KEY  (CoffeID) REFERENCES coffeeshop(Id) ")

In [19]:
d = pd.DataFrame(engine.execute("SELECT * FROM address Inner join coffeeshop on address.CoffeID = coffeeshop.Id").fetchall())
d

CoffeID      City        NeighborHood  \
0           0     تهران            زعفرانیه   
1           1     تهران             فرمانیه   
2           2     تهران        فلسطین شمالی   
3           3     تهران               فرشته   
4           4     تهران    بلوار سعادت آباد   
...       ...       ...                 ...   
1226     1226     کرمان   خیابان بهمنیار ۲۴   
1227     1227     کرمان      خیابان بهمنیار   
1228     1228     کرمان          بلوار حمزه   
1229     1229  بندرعباس         رسالت شمالی   
1230     1230  بندرعباس  خیابان رسالت جنوبی   

                                         AuxiliaryRoad    Id  \
0               آصف، انتهای کوچه حامد، نبش باغستان ششم     0   
1      خیابان لواسانی، مابین آقایی و وطن پور، پلاک 152     1   
2                       نرسیده به بلوار کشاورز،پلاک431     2   
3                                 مريم غربي،كوئين سنتر     3   
4                  نبش کوچه حق طلب،ساختمان تهران پاریس     4   
...                                                ...   ...   
1226                              انتهای محوطه پارکینگ  1226   
1227                                        بهمنیار 24  1227   
1228                                  بین کوچه 24 و 26  1228   
1229     چهارراه پردیس، ساختمان ثبت شرکت(ریاضیات خیام)  1229   
1230                        بلوار12فروردین به طرف ۹ دی  1230   

           CoffeeShopName StartAt     EndAt  CountOfFeaturs  PhoneNumber  
0                   خامیز    8:00  24:00:00               2   2126808894  
1     کافه رستوران بارمان   10:30  24:00:00               1   2122202929  
2                   سیزده    8:00  24:00:00               2   2188914292  
3                 اینساید   10:00  24:00:00               3   2122661143  
4               هوبو لانژ   14:00  24:00:00               1   9190054735  
...                   ...     ...       ...             ...          ...  
1226            پنج و نیم    8:30  24:00:00               4   9017765438  
1227                ریتون    9:00  24:00:00               4     32469231  
1228              باریستا    7:30  24:00:00               3     32457711  
1229              کنج دنج    8:00  24:00:00               2   7633740333  
1230                پیانو   11:00  24:00:00               3   9133239334  

[1231 rows x 10 columns]